- Simple forward differentiation
- Matrix calculus using Julia's generic functions and ForwardDiff
- Reverse mode and tapes

Think of $x + \epsilon$:

$f(x + \epsilon

In [1]:
immutable Dual <: Real
    value::Float64
    deriv::Float64
end

In [2]:
import Base: +, *, convert, promote_rule

In [3]:
+(a::Dual, b::Dual) = Dual(a.value+b.value, a.deriv+b.deriv)

*(a::Dual, b::Dual) = Dual(a.value*b.value, a.deriv*b.value + a.value*b.deriv)

* (generic function with 150 methods)

In [10]:
convert{T<:Union{AbstractFloat, Integer, Rational}}(::Type{Dual}, x::T) = Dual(Float64(x), 0.0)

convert (generic function with 601 methods)

In [5]:
promote_rule{T<:Real}(::Type{Dual}, ::Type{T}) = Dual

promote_rule (generic function with 102 methods)

In [6]:
promote(Dual(1,2), 3.0)

(Dual(1.0,2.0),Dual(3.0,0.0))

In [7]:
convert(Dual, 3.0)

Dual(3.0,0.0)

In [11]:
Dual(1,2) + 3

Dual(4.0,2.0)

In [50]:
derivative(f, x) = f(Dual(x, 1)).deriv

derivative (generic function with 1 method)

In [15]:
derivative(x -> 3x^5+2, 2)

240.0

In [28]:
y = [1.,2]
f(x) = (y'* [x 2; 3 4] * y)[]

f (generic function with 1 method)

In [24]:
f(3)

29.0

In [29]:
deriative(x -> f(x), 10)

1.0

In [30]:
(y' * [Dual(3, 1) 2; 3 4] * y)[]

Dual(29.0,1.0)

## Gradients

In [38]:
f(xx) = ( (x, y) = xx; x^2 + x*y )

f (generic function with 1 method)

In [44]:
xx_fixed = [2., 3]

dx = 0.01

xx_new = xx_fixed + [i==1 ? dx : 0.0 for i in 1:length(xx)]

2-element Array{Float64,1}:
 2.01
 3.0 

In [46]:
(f(xx_new) - f(xx_fixed)) / dx

7.0099999999998275

In [71]:
f1(x) = f([x, xx_fixed[2]])

f1 (generic function with 1 method)

In [74]:
[1, 2.5]

2-element Array{Float64,1}:
 1.0
 2.5

In [73]:
([Dual(3,1), xx_fixed[2]])  # automatic promotion

2-element Array{Dual,1}:
 Dual(3.0,1.0)
 Dual(3.0,0.0)

In [49]:
derivative(f1, xx_fixed[1])

7.0

In [84]:


replace_ith(xx, i, x_i) = (xx_new = copy(xx); xx_new[i] = x_i; xx_new)

function ∇(f, xx)  # can add  partials = zeros(n), xnew = similar(xx, Dual)  optional arguments
    n = length(xx)
    
    partials = zeros(n)
    xnew = similar(xx, Dual)
    copy!(xnew, xx)
    for i in 1:n
        xnew[i] = Dual(xx[i], 1.0)
        partials[i] = f(xnew).deriv
        xnew[i] = Dual(xx[i], 0.0)
    end
    
#     for i in 1:n
        
#         f_i = (x, i) -> begin
#             xnew = similar(xx, Dual) 
#             copy!(xnew, xx)
#             xnew[i] = xx[i]
#             #             [j == i ? x_i : xx[j] for j in 1:length(xx)]  # replace only ith entry of xx to make xnew
#             @show xnew
#             f(xnew)
#         end
                        
#         partials[i] = derivative(x -> f_i(x, i), xx[i])
#     end
    
    return partials
    
end

∇ (generic function with 1 method)

In [85]:
replace_ith([1, 2], 2, 3)

2-element Array{Int64,1}:
 1
 3

In [86]:
∇(f, [1., 2])

2-element Array{Float64,1}:
 4.0
 1.0

Note that my version (commented) was *very inefficient*.

# Parameterize

In [1]:
import Base: promote_type, promote_rule, convert, zero, +, *

In [2]:
immutable Dual{T} <: Real
    value::T
    deriv::T
end

In [3]:
zero{T}(::Dual{T}) = Dual(zero(T), zero(T))

zero (generic function with 15 methods)

In [4]:
promote_rule{T<:Union{AbstractFloat, Integer}, S<:Real}(::Type{Dual{S}}, ::Type{T}) = Dual{promote_type(S,T)}

promote_rule (generic function with 102 methods)

In [5]:
convert{T<:Union{AbstractFloat, Integer, Rational}, S<:Real}(::Type{Dual{S}}, x::T) = Dual(convert(S, x), zero(S))

convert (generic function with 600 methods)

In [6]:
Dual(1, 2)

Dual{Int64}(1,2)

In [7]:
promote(Dual(1., 2.), 3.0)

(Dual{Float64}(1.0,2.0),Dual{Float64}(3.0,0.0))

In [10]:
+(a::Dual, b::Dual) = Dual(a.value+b.value, a.deriv+b.deriv)

*(a::Dual, b::Dual) = Dual(a.value*b.value, a.deriv*b.value + a.value*b.deriv)

* (generic function with 150 methods)

In [11]:
Dual(1,2) + 3

Dual{Int64}(4,2)

In [12]:
derivative(f, x) = f(Dual(x, 1)).deriv

derivative (generic function with 1 method)

In [13]:
derivative(x -> 3x^5+2, 2)

240

In [14]:
(x->15x^4)(2)

240

In [15]:
y = [1.,2]
f(x) = (y'* [x 2; 3 4] * y)[]

f (generic function with 1 method)

In [16]:
f(3)

29.0

In [30]:
(y' * [Dual(3, 1) 2; 3 4] * y)[]

Dual(29.0,1.0)

In [20]:
Dual{T, S}(a::T, b::S) = Dual(promote(a, b)...)

Dual{T}

In [29]:
deriv2(f, x) = f(Dual(Dual(x, 1), 1))

deriv2 (generic function with 1 method)

In [30]:
deriv2(x->x^3, 2.0)

Dual{Dual{Float64}}(Dual{Float64}(8.0,12.0),Dual{Float64}(12.0,12.0))

In [24]:
deriv(f, x) = f(Dual(x, 1))

deriv (generic function with 1 method)

In [26]:
deriv(x->x^2, 3.0)

Dual{Float64}(9.0,6.0)

## Optimization

In [31]:
x = rand(3)
y = rand(3)

f(W) = W*x - y

f (generic function with 1 method)

In [32]:
using ForwardDiff

In [33]:
ForwardDiff.jacobian(f, rand(3,3))

3×9 Array{Float64,2}:
 0.148518  0.0       0.0       0.984806  …  0.457865  0.0       0.0     
 0.0       0.148518  0.0       0.0          0.0       0.457865  0.0     
 0.0       0.0       0.148518  0.0          0.0       0.0       0.457865

In [34]:
f(W) = (a = W*x - y; dot(a, a))

f (generic function with 1 method)

In [45]:
W0 = rand(3, 3)
grad = ForwardDiff.gradient(f, W0)

3×3 Array{Float64,2}:
  0.16606    1.10113    0.511947
 -0.130589  -0.865921  -0.402593
  0.136358   0.904175   0.420378

In [48]:
2*(W0*x-y)*x' == grad

true